In [4]:
# The following is from "big run"
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np
from lib.dblib import Database

db = dblib.Database()

# Setup
lsa_np = np.load('../data/parsed/lsa_output.npy')
metadata = db.df_from_table('emails')


ImportError: No module named 'sqlalchemy'

In [ ]:

metadata = metadata.loc[metadata['Scenario'] == '401']
metadata = metadata.reset_index(drop=True)
lsa_df = pd.DataFrame(lsa_np)
df = pd.concat([metadata, lsa_df], axis=1, join_axes=[metadata.index])
df = df.loc[df['Relevant'] != '-1']
df = df.reset_index(drop=True)
cat_features = ['To','From']
features = list(range(100))
features.extend(cat_features + ['Date'])
# features.extend(cat_features + ['ID'])

df = df[features + ['Label'] + ['Relevant'] + ['ID']]
# df = df[features + ['ID']]

# Built-in incremental learning vs trees training on larger initial sizes

## Initializing control variables

In [1]:
# now using params from sweep

n_trees = 32
tree_depth = 70
random_seed = 42
n_max_features = 90
cat_features = ['To', 'From']

## Forests Trained on increasing datasets

In [2]:
forest_100 = RNF(df[:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
forest_200 = RNF(df[:200], n_trees, tree_depth, random_seed, n_max_features, 200, cat_features)
forest_300 = RNF(df[:300], n_trees, tree_depth, random_seed, n_max_features, 300, cat_features)
forest_400 = RNF(df[:400], n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
forest_500 = RNF(df[:500], n_trees, tree_depth, random_seed, n_max_features, 500, cat_features)
incremental_forests = [forest_100, forest_200, forest_300, forest_400, forest_500]

NameError: name 'RNF' is not defined

In [ ]:
for forest in incremental_forests:
    forest.fit_parallel()

In [3]:
# evaluation
for forest in incremental_forests:
    print(evalStats(forest.predict_parallel(df[-100:])[1], df[-100:]), end='\n\n')

NameError: name 'incremental_forests' is not defined

100, 200, ..., 500 document - trained forests
Param

##  Incremental Forests

In [ ]:
incremental_forest = RNF(df[0:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)

In [ ]:
incremental_forest.fit_parallel()
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

when splitting categorically, no best address found: [56 34 39]
(0.45714285714285713, 0.9411764705882353, 0.8, 0.6153846153846154)


In [ ]:
incremental_forest.update(df[100:200])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[200:300])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[300:400])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[400:500])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

32
(0.7714285714285715, 0.9, 0.89, 0.8307692307692307)
23
(0.8571428571428571, 0.8571428571428571, 0.9, 0.8571428571428571)
28
(0.7714285714285715, 0.84375, 0.87, 0.8059701492537314)
19


In [6]:
incremental_forest.train_data.shape[0]

500

In [12]:
forest_500.train_data.shape[0]

500

# Testing limited core usage

In [ ]:
f = RNF(df[0:500], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
f.fit_parallel()

In [ ]:
f.trees[20].visualize()

# Testing 400 incremented vs 400 initial trained

In [5]:
# incremental forest initially trained on 100 rows
inc_forest = RNF(df[0:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
inc_forest.fit_parallel();

In [6]:
init_forest = RNF(df[:-100], n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
init_forest.fit_parallel();